In [ ]:
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 982.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

In [ ]:
!pip install -U -q PyDrive

In [ ]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.67.1
    Uninstalling grpcio-1.67.1:
      Successfully uninstalled grpcio-1.67.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but y

In [ ]:
import os
import torch
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision import transforms
import numpy as np

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Step 1: Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Define the path to the file in your Google Drive
zip_file_path = '/content/drive/MyDrive/archive_1.zip'
extract_to_path = '/content/data/'  # Local directory in Colab

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Data has been extracted to {extract_to_path}")



Data has been extracted to /content/data/


In [ ]:
# Load pre-trained FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
# Define image preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
def generate_embedding(image_path):
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension

    # Generate embedding
    with torch.no_grad():
        embedding = model(img_tensor)
    return embedding.squeeze().numpy()  # Convert to numpy array

def save_embeddings():
    data_dir = 'data/celebrity_images'
    embedding_dir = 'data/embeddings'
    os.makedirs(embedding_dir, exist_ok=True)

    # Process each celebrity folder
    for celebrity_name in os.listdir(data_dir):
        celebrity_folder = os.path.join(data_dir, celebrity_name)
        embeddings = []

        for img_name in os.listdir(celebrity_folder):
            img_path = os.path.join(celebrity_folder, img_name)
            embedding = generate_embedding(img_path)  # Pass img_path here
            embeddings.append(embedding)

        # Save embeddings as a .npy file
        embeddings = np.stack(embeddings)
        np.save(os.path.join(embedding_dir, f"{celebrity_name}.npy"), embeddings)
        print(f"Saved embeddings for {celebrity_name}")

if __name__ == '__main__':
    save_embeddings()

Saved embeddings for pins_Cristiano Ronaldo
Saved embeddings for pins_jeff bezos
Saved embeddings for pins_Lionel Messi
Saved embeddings for pins_Megan Fox
Saved embeddings for pins_Dwayne Johnson
Saved embeddings for pins_Josh Radnor
Saved embeddings for pins_Penn Badgley
Saved embeddings for pins_Sarah Wayne Callies
Saved embeddings for pins_Ben Affleck
Saved embeddings for pins_Katherine Langford
Saved embeddings for pins_Mark Zuckerberg
Saved embeddings for pins_Robert De Niro
Saved embeddings for pins_Tom Hiddleston
Saved embeddings for pins_Avril Lavigne
Saved embeddings for pins_Bill Gates
Saved embeddings for pins_Brian J. Smith
Saved embeddings for pins_Hugh Jackman
Saved embeddings for pins_Rami Malek
Saved embeddings for pins_Jason Momoa
Saved embeddings for pins_Henry Cavil
Saved embeddings for pins_Miley Cyrus
Saved embeddings for pins_Sophie Turner
Saved embeddings for pins_Logan Lerman
Saved embeddings for pins_elon musk
Saved embeddings for pins_Maria Pedraza
Saved embe

In [ ]:
from qdrant_client import QdrantClient
import numpy as np
import os
from qdrant_client.models import Distance, VectorParams
from qdrant_client.http.models import VectorParams
# Replace with your Qdrant cloud URL or Docker container IP
qdrant_url = ""
api_key = ""

# Initialize Qdrant client (use the URL for remote instances)
client = QdrantClient(
    url=qdrant_url,
    api_key=api_key  # If using cloud, include API key for authentication
)

# Define the collection name and check if it exists
collection_name = "celebrity_embeddings_3"
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            "default": VectorParams(size=512, distance=Distance.COSINE)
        }
    )
    print(f"Created collection: {collection_name}")
else:
    print(f"Collection '{collection_name}' already exists.")

def load_embeddings_to_qdrant():
    embedding_dir = 'data/embeddings'  # Directory with your .npy files
    point_id = 0  # Initialize a unique integer ID for each embedding

    for celebrity_file in os.listdir(embedding_dir):
        celebrity_name = os.path.splitext(celebrity_file)[0]
        embeddings = np.load(os.path.join(embedding_dir, celebrity_file))

        points = [
            {
                "id": point_id + i,  # Use a unique integer ID
                "vector": {"default": embedding.tolist()},  # Specify the vector name "default"
                "payload": {"celebrity": celebrity_name}
            }
            for i, embedding in enumerate(embeddings)
        ]

        # Update point_id to continue incrementing across files
        point_id += len(points)

        client.upsert(
            collection_name=collection_name,
            points=points
        )
        print(f"Uploaded {len(points)} embeddings for {celebrity_name}")

if __name__ == '__main__':
    load_embeddings_to_qdrant()


Collection 'celebrity_embeddings_3' already exists.
Uploaded 203 embeddings for pins_Anne Hathaway
Uploaded 179 embeddings for pins_Hugh Jackman
Uploaded 141 embeddings for pins_Dwayne Johnson
Uploaded 166 embeddings for pins_Chris Evans
Uploaded 181 embeddings for pins_Tom Hiddleston
Uploaded 146 embeddings for pins_Dominic Purcell
Uploaded 158 embeddings for pins_Elizabeth Lail
Uploaded 117 embeddings for pins_Josh Radnor
Uploaded 178 embeddings for pins_Rebecca Ferguson
Uploaded 175 embeddings for pins_Morena Baccarin
Uploaded 133 embeddings for pins_Nadia Hilker
Uploaded 212 embeddings for pins_Logan Lerman
Uploaded 98 embeddings for pins_Cristiano Ronaldo
Uploaded 122 embeddings for pins_Maria Pedraza
Uploaded 127 embeddings for pins_Inbar Lavi
Uploaded 226 embeddings for pins_Katherine Langford
Uploaded 196 embeddings for pins_Andy Samberg
Uploaded 138 embeddings for pins_Zendaya
Uploaded 201 embeddings for pins_scarlett johansson
Uploaded 122 embeddings for pins_Bill Gates
Uploa

In [ ]:
# upload_embeddings.py
from qdrant_client import QdrantClient
import numpy as np
import os
from qdrant_client.models import Distance, VectorParams, PointStruct

# Qdrant connection configuration
qdrant_url = "https://34ef3520-a13f-4224-a4c5-f12c09b9a937.us-east4-0.gcp.cloud.qdrant.io"
api_key = "DuT8NuJMBhg18S7HE6GC2tNZ1TYt-cg5iiciJML808W9OGo2uqcF0g"

# Initialize Qdrant client
client = QdrantClient(
    url=qdrant_url,
    api_key=api_key
)

# Collection configuration
collection_name = "celebrity_embeddings_3"
vector_size = 512

def ensure_collection():
    """Create collection if it doesn't exist"""
    if not client.collection_exists(collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
        )
        print(f"Created collection: {collection_name}")
    else:
        print(f"Collection '{collection_name}' already exists.")

def load_embeddings_to_qdrant():
    """Load embeddings from .npy files and upload to Qdrant"""
    embedding_dir = 'data/embeddings'
    point_id = 0

    for celebrity_file in os.listdir(embedding_dir):
        celebrity_name = os.path.splitext(celebrity_file)[0]
        embeddings = np.load(os.path.join(embedding_dir, celebrity_file))

        # Convert embeddings to float32 and handle NaN values
        embeddings = np.nan_to_num(embeddings.astype(np.float32))

        points = [
            PointStruct(
                id=point_id + i,
                vector=embedding.tolist(),  # Convert numpy array to list
                payload={"celebrity": celebrity_name}
            )
            for i, embedding in enumerate(embeddings)
        ]

        # Update point_id for next batch
        point_id += len(points)

        # Upload points in batches
        batch_size = 100
        for i in range(0, len(points), batch_size):
            batch = points[i:i + batch_size]
            client.upsert(
                collection_name=collection_name,
                points=batch
            )

        print(f"Uploaded {len(points)} embeddings for {celebrity_name}")

if __name__ == '__main__':
    ensure_collection()
    load_embeddings_to_qdrant()

Created collection: celebrity_embeddings_3
Uploaded 203 embeddings for pins_Anne Hathaway
Uploaded 179 embeddings for pins_Hugh Jackman
Uploaded 141 embeddings for pins_Dwayne Johnson
Uploaded 166 embeddings for pins_Chris Evans
Uploaded 181 embeddings for pins_Tom Hiddleston
Uploaded 146 embeddings for pins_Dominic Purcell
Uploaded 158 embeddings for pins_Elizabeth Lail
Uploaded 117 embeddings for pins_Josh Radnor
Uploaded 178 embeddings for pins_Rebecca Ferguson
Uploaded 175 embeddings for pins_Morena Baccarin
Uploaded 133 embeddings for pins_Nadia Hilker
Uploaded 212 embeddings for pins_Logan Lerman
Uploaded 98 embeddings for pins_Cristiano Ronaldo
Uploaded 122 embeddings for pins_Maria Pedraza
Uploaded 127 embeddings for pins_Inbar Lavi
Uploaded 226 embeddings for pins_Katherine Langford
Uploaded 196 embeddings for pins_Andy Samberg
Uploaded 138 embeddings for pins_Zendaya
Uploaded 201 embeddings for pins_scarlett johansson
Uploaded 122 embeddings for pins_Bill Gates
Uploaded 189 e